## Stream  직접 구현해보기

- OpenAI의 기본 모듈을 이용하면 Stream이 얼마나 까다로운지 잘 분간이 안감
- 실제로 어떤 형태로 구현되는지 직접 코드를 작성해보면 내부 논리/로직을 파악할 수 있음
- Stream은 generator의 일종, 그래서 이 Stream object를 for chunk in stream 형태로 순회를 해줘야 그 안에 있는 정보를 빼올 수 있음


In [2]:
from openai import OpenAI


## Stream을 만드는 여러가지 다양한 방법
## 쉬운 방법 : 그냥 OpenAI를 이용하기

OPENAI_API_KEY=""


## 기본 OpenAI Object을 잉용하는 경우
client = OpenAI(api_key=OPENAI_API_KEY)

stream = client.chat.completions.create(
    model="gpt-4o-mini",

    messages=[{"role": "user", "content": "Say this is a test please give me a thre paragraph"}],
    stream=True,
)
## stream itself is a generator so we can do chunk in stream
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")



Certainly! Here’s a three-paragraph text for you:

---

In today's fast-paced world, the significance of mindfulness has become increasingly apparent. Mindfulness is the practice of being present and fully engaged in the moment, without judgment or distraction. This approach to life has roots in ancient meditation traditions, but its principles have been adopted in modern psychology and wellness practices. By cultivating mindfulness, individuals can alleviate stress, enhance emotional well-being, and develop a greater appreciation for the small joys in life.

Incorporating mindfulness into daily routines does not have to be complicated. Simple practices, such as mindful breathing or taking a few moments to observe one's surroundings, can create a profound impact. Many find that dedicating a few minutes each day to mindfulness exercises helps ground them in the present, reducing anxiety about the future and regrets about the past. These small acts of awareness can lead to significant ch

In [6]:



# In Python’s typing system, the Generator type from the typing module is used to specify the type of generator functions. The Generator type takes three type parameters:

# 	1.	The type of values the generator yields.
# 	2.	The type of values that can be sent to the generator (using the send method).
# 	3.	The return type of the generator when it terminates.

# For the stream_openai_chat function, these types are:

# 	1.	str: The generator yields strings, as it produces chunks of the response content.
# 	2.	None: The generator does not accept any values via the send method.
# 	3.	None: The generator does not return any value when it terminates.

# So the correct type hint for this generator function would be Generator[str, None, None]. Here’s the updated code with this explanation:



import requests
import json
from typing import List, Dict, Generator

def stream_openai_chat(messages: List[Dict[str, str]]) -> Generator[str, None, None]:
    model = "gpt-4o-mini"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {OPENAI_API_KEY}',
    }

    json_data = {
        'model': model,
        'stream': True,
        'messages': messages,
    }

    response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, json=json_data, stream=True)

    if response.status_code == 200:
        for chunk in response.iter_lines():
            if chunk:
                chunk_data = chunk.decode('utf-8')
                if chunk_data.startswith('data:'):
                    chunk_data = chunk_data[len('data:'):].strip()
                    if chunk_data != '[DONE]':
                        try:
                            chunk_json = json.loads(chunk_data)
                            if 'choices' in chunk_json:
                                delta = chunk_json['choices'][0]['delta']
                                if 'content' in delta:
                                    yield delta['content']
                        except json.JSONDecodeError:
                            pass
    else:
        yield f"Request failed with status code {response.status_code}"

# Example usage
messages = [
    {
        'role': 'user',
        'content': 'Say this is a test please give me a three paragraph text.',
    },
]

for content in stream_openai_chat(messages):
    print(content, end='')

This is a test. In a world where technology continuously evolves, the way we communicate and interact has transformed dramatically. Gone are the days when messages took days or weeks to reach their destinations; now, we are connected instantaneously across the globe. This shift not only enhances personal relationships but also influences business operations, social movements, and education. With the click of a button, information can be shared, ideas can be exchanged, and communities can be formed, transcending geographical boundaries.

However, this rapid development in communication technology is not without its challenges. The rise of social media has ushered in an era of information overload, where individuals are bombarded with opinions, news, and ads at an unprecedented scale. The potential for misinformation and echo chambers poses a significant risk, as people may find it challenging to discern credible sources from unreliable ones. Mental health implications also arise as the 